In [1]:
from __future__ import division, absolute_import

import os
import sys
import random

import numpy as np
import datetime

#root
absPath = '/home/angela/padding_EBI/'
sys.path.insert(0, absPath)

#from src.Target import Target

np.random.seed(8)
random.seed(8)

from src.preprocessing import *
from src.model_architecture import *
from src.training_model import *
from src.postprocessing import *
from src.comparing_results import *
#from src.callbacks import *
%matplotlib inline

Using TensorFlow backend.


### Defining parameters

In [2]:
list_paddings = [
    'post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
                'rnd_padding', 
                 'aug_padding',
                "zoom_padding"
]
#list_paddings = ['post_padding', 'pre_padding', 'mid_padding', 'strf_padding', 'ext_padding', 
#                 'rnd_padding']
#list_paddings = ["aug_padding"]

#hierarchy of folders: annotation/dataset/architecture/n_neurs/task/padding
folder = 'EC_number/archaea/3denses/bio_neurons2/' #'EC_number/archaea/rnn_conv/256rnn/' #3denses/bio_neurons/'
#'EC_number/archaea/stack_conv/10filts_sizeJurtz/'
#1conv/64filts_size5/' #'EC_number/archaea/3denses/bio_neurons/'
column = "EC number"

n_folds = 10

dicti = creating_dict()

max_lenn = 1000

n_class = 7 #number of classes to output
drop_per = 0.2 #Input dropout 
n_neur = [313,76]
drop_hid = 0.5
dict_size = len(dicti)
final_act = "softmax"
n_filt = 10 #64 #None
kernel_size = [5] #[1,3,5,9,15]#[5] #None
pool_size = 10 #None
n_hid = 256 #None

batch_size = 54
epochss = 200

callbacks_list = [None, None, None, None, False, False, False, False]
task = "task2/"
architecture = "only_denses"
#architecture = "conv_dense"
#architecture = "rnn_conv"
#architecture = "stack_conv"

In [3]:
lr=0.0001
optimizer = Adam(lr=lr)

### Loading stuff

In [4]:
###LOADING ENZYME INDICES BECAUSE THIS IS TASK 2

file_idcs_aug = os.path.join(absPath, 'data/', folder, 'idcs_aug_enz.pickle')

with open(file_idcs_aug, "rb") as input_file:
    k_aug_indices = pickle.load(input_file)
    
file_idcs = os.path.join(absPath, 'data/', folder, 'idcs_data_enz.pickle')

with open(file_idcs, "rb") as input_file:
    splitting_sets = pickle.load(input_file)

# Defining Deep Learning model - Task 2

In [5]:
model = building_2dense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 'softmax', folder)
#model = building_1convdense_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, kernel_size, final_act, folder)
#model = building_convrnn_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, 
#                                 n_filt, kernel_size, pool_size, n_hid, final_act, folder, optimizer)
#model = building_stackconv_model_task2(max_lenn, dict_size, n_neur, n_class, drop_per, drop_hid, n_filt, 
#                                       kernel_size, pool_size, final_act, folder, optimizer)

generators_dict = {} 
for model_type in list_paddings:
    generators_dict[model_type] = trainval_generators(splitting_sets, k_aug_indices, model_type, 
                                                      folder, batch_size, 'labels_task2', True)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
dense_1 (Dense)              (None, 1000, 313)         8451      
_________________________________________________________________
dropout_2 (Dropout)          (None, 1000, 313)         0         
_________________________________________________________________
dense_2 (Dense)              (None, 1000, 76)          23864     
_________________________________________________________________
dropout_3 (Dropout)          (None, 1000, 76)          0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 76000)             0         
__________

### Training model

In [6]:
for model_type in list_paddings:
    generators = generators_dict[model_type]
    for idx,i in enumerate(generators):
        if model_type == "aug_padding":
            i_train, i_val, i_test = k_aug_indices[idx]
        else: 
            i_train, i_val, i_test = splitting_sets[idx]
        len_train, len_val, len_test = len(i_train), len(i_val), len(i_test)
        train_generator, val_generator = i
        model_training(model_type, folder, task, idx, callbacks_list, train_generator, 
                        val_generator, architecture, max_lenn, dict_size, batch_size, 
                        n_neur, n_class, drop_per, drop_hid, final_act, epochss, 
                        len_train, len_val,
                        n_filt = n_filt, kernel_size=kernel_size, pool_size=pool_size,
                      nhid=n_hid, optimizer=optimizer)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 1000, 26)          0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1000, 26)          0         
_________________________________________________________________
dense_4 (Dense)              (None, 1000, 313)         8451      
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000, 313)         0         
_________________________________________________________________
dense_5 (Dense)              (None, 1000, 76)          23864     
_________________________________________________________________
dropout_6 (Dropout)          (None, 1000, 76)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 76000)             0         
__________

138/138 [==============================] - 3s 25ms/step - loss: 0.0708 - acc: 0.4634 - val_loss: 0.0166 - val_acc: 0.2050

Epoch 00034: val_acc did not improve from 0.23563
Epoch 35/200
138/138 [==============================] - 3s 24ms/step - loss: 0.0644 - acc: 0.4552 - val_loss: 0.0207 - val_acc: 0.2235

Epoch 00035: val_acc did not improve from 0.23563
Epoch 36/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0693 - acc: 0.4634 - val_loss: 0.0198 - val_acc: 0.2050

Epoch 00036: val_acc did not improve from 0.23563
Epoch 37/200
138/138 [==============================] - 3s 24ms/step - loss: 0.0607 - acc: 0.4509 - val_loss: 0.0167 - val_acc: 0.2139

Epoch 00037: val_acc did not improve from 0.23563
Epoch 38/200
138/138 [==============================] - 3s 24ms/step - loss: 0.0683 - acc: 0.4572 - val_loss: 0.0157 - val_acc: 0.2082

Epoch 00038: val_acc did not improve from 0.23563
Epoch 39/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0635

138/138 [==============================] - 3s 24ms/step - loss: 0.0421 - acc: 0.4716 - val_loss: 0.0139 - val_acc: 0.2031

Epoch 00078: val_acc did not improve from 0.23563
Epoch 79/200
138/138 [==============================] - 3s 24ms/step - loss: 0.0436 - acc: 0.4560 - val_loss: 0.0134 - val_acc: 0.2133

Epoch 00079: val_acc did not improve from 0.23563
Epoch 80/200
138/138 [==============================] - 3s 24ms/step - loss: 0.0429 - acc: 0.4681 - val_loss: 0.0122 - val_acc: 0.2107

Epoch 00080: val_acc did not improve from 0.23563
Epoch 81/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0436 - acc: 0.4640 - val_loss: 0.0087 - val_acc: 0.2050

Epoch 00081: val_acc did not improve from 0.23563
Epoch 82/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0470 - acc: 0.4626 - val_loss: 0.0126 - val_acc: 0.2095

Epoch 00082: val_acc did not improve from 0.23563
Epoch 83/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0401

138/138 [==============================] - 3s 25ms/step - loss: 0.0409 - acc: 0.4669 - val_loss: 0.0077 - val_acc: 0.2056

Epoch 00122: val_acc did not improve from 0.23563
Epoch 123/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0375 - acc: 0.4626 - val_loss: 0.0105 - val_acc: 0.2056

Epoch 00123: val_acc did not improve from 0.23563
Epoch 124/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0380 - acc: 0.4608 - val_loss: 0.0151 - val_acc: 0.2088

Epoch 00124: val_acc did not improve from 0.23563
Epoch 125/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0404 - acc: 0.4685 - val_loss: 0.0068 - val_acc: 0.2050

Epoch 00125: val_acc did not improve from 0.23563
Epoch 126/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0376 - acc: 0.4630 - val_loss: 0.0115 - val_acc: 0.2031

Epoch 00126: val_acc did not improve from 0.23563
Epoch 127/200
138/138 [==============================] - 3s 25ms/step - loss: 0

138/138 [==============================] - 3s 25ms/step - loss: 0.0368 - acc: 0.4651 - val_loss: 0.0084 - val_acc: 0.2043

Epoch 00166: val_acc did not improve from 0.23563
Epoch 167/200
138/138 [==============================] - 3s 24ms/step - loss: 0.0369 - acc: 0.4630 - val_loss: 0.0072 - val_acc: 0.2011

Epoch 00167: val_acc did not improve from 0.23563
Epoch 168/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0377 - acc: 0.4593 - val_loss: 0.0112 - val_acc: 0.2133

Epoch 00168: val_acc did not improve from 0.23563
Epoch 169/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0369 - acc: 0.4685 - val_loss: 0.0113 - val_acc: 0.2050

Epoch 00169: val_acc did not improve from 0.23563
Epoch 170/200
138/138 [==============================] - 3s 25ms/step - loss: 0.0356 - acc: 0.4611 - val_loss: 0.0115 - val_acc: 0.2005

Epoch 00170: val_acc did not improve from 0.23563
Epoch 171/200
138/138 [==============================] - 3s 25ms/step - loss: 0

...

138/138 [==============================] - 4s 27ms/step - loss: 0.1620 - acc: 0.4334 - val_loss: 0.0411 - val_acc: 0.2222

Epoch 00013: val_acc did not improve from 0.22478
Epoch 14/200
138/138 [==============================] - 4s 27ms/step - loss: 0.1276 - acc: 0.4487 - val_loss: 0.0336 - val_acc: 0.2241

Epoch 00014: val_acc did not improve from 0.22478
Epoch 15/200
138/138 [==============================] - 4s 28ms/step - loss: 0.1124 - acc: 0.4516 - val_loss: 0.0270 - val_acc: 0.2241

Epoch 00015: val_acc did not improve from 0.22478
Epoch 16/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0954 - acc: 0.4567 - val_loss: 0.0225 - val_acc: 0.2216

Epoch 00016: val_acc did not improve from 0.22478
Epoch 17/200
138/138 [==============================] - 4s 27ms/step - loss: 0.0837 - acc: 0.4556 - val_loss: 0.0204 - val_acc: 0.2267

Epoch 00017: val_acc improved from 0.22478 to 0.22669, saving model to /home/angela/padding_EBI/data/checkpoint/EC_number/archaea/3de

138/138 [==============================] - 4s 27ms/step - loss: 0.0290 - acc: 0.4746 - val_loss: 0.0051 - val_acc: 0.2063

Epoch 00056: val_acc did not improve from 0.22669
Epoch 57/200
138/138 [==============================] - 4s 27ms/step - loss: 0.0275 - acc: 0.4605 - val_loss: 0.0067 - val_acc: 0.2069

Epoch 00057: val_acc did not improve from 0.22669
Epoch 58/200
138/138 [==============================] - 4s 27ms/step - loss: 0.0305 - acc: 0.4671 - val_loss: 0.0048 - val_acc: 0.2075

Epoch 00058: val_acc did not improve from 0.22669
Epoch 59/200
138/138 [==============================] - 4s 27ms/step - loss: 0.0295 - acc: 0.4670 - val_loss: 0.0052 - val_acc: 0.2082

Epoch 00059: val_acc did not improve from 0.22669
Epoch 60/200
138/138 [==============================] - 4s 27ms/step - loss: 0.0270 - acc: 0.4681 - val_loss: 0.0053 - val_acc: 0.2152

Epoch 00060: val_acc did not improve from 0.22669
Epoch 61/200
138/138 [==============================] - 4s 27ms/step - loss: 0.0297

138/138 [==============================] - 4s 28ms/step - loss: 0.0281 - acc: 0.4635 - val_loss: 0.0053 - val_acc: 0.2114

Epoch 00100: val_acc did not improve from 0.22669
Epoch 101/200
138/138 [==============================] - 4s 27ms/step - loss: 0.0252 - acc: 0.4713 - val_loss: 0.0046 - val_acc: 0.2063

Epoch 00101: val_acc did not improve from 0.22669
Epoch 102/200
138/138 [==============================] - 4s 27ms/step - loss: 0.0271 - acc: 0.4670 - val_loss: 0.0058 - val_acc: 0.2075

Epoch 00102: val_acc did not improve from 0.22669
Epoch 103/200
138/138 [==============================] - 4s 27ms/step - loss: 0.0250 - acc: 0.4699 - val_loss: 0.0049 - val_acc: 0.2088

Epoch 00103: val_acc did not improve from 0.22669
Epoch 104/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0246 - acc: 0.4711 - val_loss: 0.0046 - val_acc: 0.2075

Epoch 00104: val_acc did not improve from 0.22669
Epoch 105/200
138/138 [==============================] - 4s 27ms/step - loss: 0

138/138 [==============================] - 4s 28ms/step - loss: 0.0235 - acc: 0.4662 - val_loss: 0.0047 - val_acc: 0.2069

Epoch 00144: val_acc did not improve from 0.22669
Epoch 145/200
138/138 [==============================] - 4s 27ms/step - loss: 0.0226 - acc: 0.4666 - val_loss: 0.0046 - val_acc: 0.2069

Epoch 00145: val_acc did not improve from 0.22669
Epoch 146/200
138/138 [==============================] - 4s 27ms/step - loss: 0.0239 - acc: 0.4673 - val_loss: 0.0047 - val_acc: 0.2101

Epoch 00146: val_acc did not improve from 0.22669
Epoch 147/200
138/138 [==============================] - 4s 27ms/step - loss: 0.0228 - acc: 0.4702 - val_loss: 0.0061 - val_acc: 0.2069

Epoch 00147: val_acc did not improve from 0.22669
Epoch 148/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0247 - acc: 0.4679 - val_loss: 0.0046 - val_acc: 0.2075

Epoch 00148: val_acc did not improve from 0.22669
Epoch 149/200
138/138 [==============================] - 4s 27ms/step - loss: 0

138/138 [==============================] - 4s 28ms/step - loss: 0.0229 - acc: 0.4681 - val_loss: 0.0041 - val_acc: 0.2082

Epoch 00188: val_acc did not improve from 0.22669
Epoch 189/200
138/138 [==============================] - 4s 27ms/step - loss: 0.0201 - acc: 0.4658 - val_loss: 0.0050 - val_acc: 0.2095

Epoch 00189: val_acc did not improve from 0.22669
Epoch 190/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0212 - acc: 0.4695 - val_loss: 0.0049 - val_acc: 0.2075

Epoch 00190: val_acc did not improve from 0.22669
Epoch 191/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0214 - acc: 0.4658 - val_loss: 0.0047 - val_acc: 0.2082

Epoch 00191: val_acc did not improve from 0.22669
Epoch 192/200
138/138 [==============================] - 4s 28ms/step - loss: 0.0215 - acc: 0.4647 - val_loss: 0.0056 - val_acc: 0.2082

Epoch 00192: val_acc did not improve from 0.22669
Epoch 193/200
138/138 [==============================] - 4s 28ms/step - loss: 0